# LangSmith Walkthrough

LangChain makes it easy to prototype LLM applications and Agents. However, delivering LLM applications to production can be deceptively difficult. You will likely have to heavily customize and iterate on your prompts, chains, and other components to create a high-quality product.

To aid in this process, we've launched LangSmith, a unified platform for debugging, testing, and monitoring your LLM applications.

When might this come in handy? You may find it useful when you want to:

- Quickly debug a new chain, agent, or set of tools
- Visualize how components (chains, llms, retrievers, etc.) relate and are used
- Evaluate different prompts and LLMs for a single component
- Run a given chain several times over a dataset to ensure it consistently meets a quality bar
- Capture usage traces and using LLMs or analytics pipelines to generate insights

## Prerequisites

**[Create a LangSmith account](https://smith.langchain.com/) and create an API key (see bottom left corner). Familiarize yourself with the platform by looking through the [docs](https://docs.smith.langchain.com/)**

Note LangSmith is in closed beta; we're in the process of rolling it out to more users. However, you can fill out the form on the website for expedited access.

Now, let's get started!

## Log runs to LangSmith

First, configure your environment variables to tell LangChain to log traces. This is done by setting the `LANGCHAIN_TRACING_V2` environment variable to true.
You can tell LangChain which project to log to by setting the `LANGCHAIN_PROJECT` environment variable (if this isn't set, runs will be logged to the `default` project). This will automatically create the project for you if it doesn't exist. You must also set the `LANGCHAIN_ENDPOINT` and `LANGCHAIN_API_KEY` environment variables.

For more information on other ways to set up tracing, please reference the [LangSmith documentation](https://docs.smith.langchain.com/docs/).

**NOTE:** You must also set your `OPENAI_API_KEY` environment variables in order to run the following tutorial.

**NOTE:** You can only access an API key when you first create it. Keep it somewhere safe.

**NOTE:** You can also use a context manager in python to log traces using
```python
from langchain.callbacks.manager import tracing_v2_enabled

with tracing_v2_enabled(project_name="My Project"):
    agent.run("How many people live in canada as of 2023?")
```

However, in this example, we will use environment variables.

In [19]:
# %pip install -U langchain langsmith langchainhub --quiet
# %pip install openai pandas duckduckgo-search --quiet

In [20]:
import os
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "ls__16337204e15f49cc90d5d4b07385c1c3"  # Update to your API key

# Used by the agent in this tutorial
# os.environ["OPENAI_API_KEY"] = "<YOUR-OPENAI-API-KEY>"

Create the langsmith client to interact with the API

In [21]:
from langsmith import Client

client = Client()

Create a LangChain component and log runs to the platform. In this example, we will create a ReAct-style agent with access to a general search tool (DuckDuckGo). The agent's prompt can be viewed in the [Hub here](https://smith.langchain.com/hub/wfh/langsmith-agent-prompt).

In [33]:
prompt.messages[0].prompt.template

'You are an expert senior software engineer. You are responsible for answering questions about LangChain. Use functions to consult the documentation before answering.'

In [36]:
from langchain.prompts import SystemMessagePromptTemplate
prompt.messages[0]  = SystemMessagePromptTemplate.from_template(
    'You are an expert senior software engineer. You are responsible for answering questions about LangChain and LangSmith, the LLM application development tools. '
    'Search the web to gather relevant information before answering.'
)

In [37]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.tools import DuckDuckGoSearchResults
from langchain.tools.render import format_tool_to_openai_function

# Fetches the latest version of this prompt
# prompt = hub.pull("wfh/langsmith-agent-prompt:latest")

llm = ChatOpenAI(
    model="gpt-3.5-turbo-16k",
    temperature=0,
)

tools = [
    DuckDuckGoSearchResults(),  # General internet search using DuckDuckGo
]

llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

runnable_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_functions(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=runnable_agent, tools=tools)

We are running the agent concurrently on multiple inputs to reduce latency. Runs get logged to LangSmith in the background so execution latency is unaffected.

In [38]:
inputs = [
    "What is LangChain?",
    "What's LangSmith?",
    "How do I use a traceable decorator?",
    "Can I trace my Llama V2 llm?",
    "Why do I have to set environment variables?",
    "How do I move my project between organizations?",
    "How do I search for a run with metadata?",
]

results = agent_executor.batch([{"input": x} for x in inputs], return_exceptions=True)

In [39]:
results[:3]

[{'input': 'What is LangChain?',
  'output': 'LangChain is not a well-known term or tool in the software development industry. It is possible that it may refer to a specific technology or framework that is not widely recognized. Without further information, it is difficult to provide a specific answer.'},
 {'input': "What's LangSmith?",
  'output': 'I apologize, but I couldn\'t find any specific information about "LangSmith" in my search results. It\'s possible that LangSmith is not a well-known tool or it may be a term that is not widely used. If you have any additional information or context about LangSmith, I would be happy to try and provide more assistance.'},
 {'input': 'How do I use a traceable decorator?',
  'output': 'To use a traceable decorator, you need to follow these steps:\n\n1. Define the decorator function: Create a function that will serve as the decorator. This function will take the target object, property name, and property descriptor as arguments.\n\n2. Implement 

Assuming you've successfully set up your environment, your agent traces should show up in the `Projects` section in the [app](https://smith.langchain.com/). Congrats!

## Evaluate Agent

In addition to logging runs, LangSmith also allows you to test and evaluate your LLM applications.

In this section, you will leverage LangSmith to create a benchmark dataset and run AI-assisted evaluators on an agent. You will do so in a few steps:

1. Create a dataset
2. Initialize a new agent to benchmark
3. Configure evaluators to grade an agent's output
4. Run the agent over the dataset and evaluate the results

### 1. Create a LangSmith dataset

Below, we use the LangSmith client to create a dataset from the input questions from above and a list labels. You will use these later to measure performance for a new agent. A dataset is a collection of examples, which are nothing more than input-output pairs you can use as test cases to your application.

For more information on datasets, including how to create them from CSVs or other files or how to create them in the platform, please refer to the [LangSmith documentation](https://docs.smith.langchain.com/).

In [ ]:
outputs = [
    "LangChain is an open-source framework for building applications using large language models. It is also the name of the company building LangSmith.",
    "LangSmith is a unified platform for debugging, testing, and monitoring language model applications and agents powered by LangChain",
    """The traceable decorator is available in the langsmith python SDK. To use, configure your environment with your API key,import the required function, decorate your function, and then call the function. Below is an example:
```python
from langsmith.run_helpers import traceable
@traceable(run_type="chain") # or "llm", etc.
def my_function(input_param):
    # Function logic goes here
    return output
result = my_function(input_param)
```""",
    "So long as you are using one of LangChain's LLM implementations, all your calls can be traced",
    "Environment variables can tell your LangChain application to perform tracing and contain the information necessary to authenticate to LangSmith. While there are other ways to connect, environment variables tend to be the simplest way to configure your application.",
    "LangSmith doesn't directly support moving projects between organizations.",
    """You can search for runs with specific metadata using the filter 'has(metadata, "<json-search>")'. An example in python is:
```python
from langsmith import Client

client = Client()
runs = list(client.list_runs(
    project_name="<your_project>",
    filter='has(metadata, '{"variant": "abc123"}')',
))```""",
]

In [ ]:
dataset_name = f"langsmith-docs-dataset-{unique_id}"

dataset = client.create_dataset(
    dataset_name, description="An example dataset of questions over the LangSmith documentation."
)

for query, answer in zip(inputs, outputs):
    client.create_example(inputs={"input": query}, outputs={"output": answer}, dataset_id=dataset.id)

### 2. Initialize a new agent to benchmark

LangSmith lets you evaluate any LLM, chain, agent, or even a custom function. Conversational agents are stateful (they have memory); to ensure that this state isn't shared between dataset runs, we will pass in a `chain_factory` (aka a `constructor`) function to initialize for each call.

In this case, we will test an agent that uses OpenAI's function calling endpoints.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, load_tools, AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.tools.render import format_tool_to_openai_function
from langchain import hub


# Since chains can be stateful (e.g. they can have memory), we provide
# a way to initialize a new chain for each row in the dataset. This is done
# by passing in a factory function that returns a new chain for each row.
def agent_factory(prompt):    
    llm_with_tools = llm.bind(
        functions=[format_tool_to_openai_function(t) for t in tools]
    )
    runnable_agent = (
            {
                "input": lambda x: x["input"],
                "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps'])
            } 
             | prompt 
             | llm_with_tools 
             | OpenAIFunctionsAgentOutputParser()
    )
    return  AgentExecutor(agent=runnable_agent, tools=tools)


### 3. Configure evaluation

Manually comparing the results of chains in the UI is effective, but it can be time consuming.
It can be helpful to use automated metrics and AI-assisted feedback to evaluate your component's performance.

Below, we will create some pre-implemented run evaluators that do the following:
- Compare results against ground truth labels.
- Measure semantic (dis)similarity using embedding distance
- Evaluate 'aspects' of the agent's response in a reference-free manner using custom criteria

For a longer discussion of how to select an appropriate evaluator for your use case and how to create your own
custom evaluators, please refer to the [LangSmith documentation](https://docs.smith.langchain.com/).


In [ ]:
from langchain.evaluation import EvaluatorType
from langchain.smith import RunEvalConfig

evaluation_config = RunEvalConfig(
    # Evaluators can either be an evaluator type (e.g., "qa", "criteria", "embedding_distance", etc.) or a configuration for that evaluator
    evaluators=[
        # Measures whether a QA response is "Correct", based on a reference answer
        # You can also select via the raw string "qa"
        EvaluatorType.QA,
        # Measure the embedding distance between the output and the reference answer
        # Equivalent to: EvalConfig.EmbeddingDistance(embeddings=OpenAIEmbeddings())
        EvaluatorType.EMBEDDING_DISTANCE,
        # Grade whether the output satisfies the stated criteria. You can select a default one such as "helpfulness" or provide your own.
        RunEvalConfig.LabeledCriteria("helpfulness"),
        # Both the Criteria and LabeledCriteria evaluators can be configured with a dictionary of custom criteria.
        RunEvalConfig.Criteria(
            {
                "fifth-grader-score": "Do you have to be smarter than a fifth grader to answer this question?"
            }
        ),
    ],
    # You can add custom StringEvaluator or RunEvaluator objects here as well, which will automatically be
    # applied to each prediction. Check out the docs for examples.
    custom_evaluators=[],
)

### 4. Run the agent and evaluators

Use the [run_on_dataset](https://api.python.langchain.com/en/latest/smith/langchain.smith.evaluation.runner_utils.run_on_dataset.html#langchain.smith.evaluation.runner_utils.run_on_dataset) (or asynchronous [arun_on_dataset](https://api.python.langchain.com/en/latest/smith/langchain.smith.evaluation.runner_utils.arun_on_dataset.html#langchain.smith.evaluation.runner_utils.arun_on_dataset)) function to evaluate your model. This will:
1. Fetch example rows from the specified dataset.
2. Run your agent (or any custom function) on each example.
3. Apply evalutors to the resulting run traces and corresponding reference examples to generate automated feedback.

The results will be visible in the LangSmith app.

In [ ]:
prompt = hub.pull("wfh/langsmith-agent-prompt:5d466cbc")
prompt.format_messages(input="foo", chat_history=[], agent_scratchpad=[])

In [10]:
len("5d466cbc")

8

In [18]:
from langchain import hub
hub.pull("wfh/langsmith-agent-prompt:798e7324")

ChatPromptTemplate(input_variables=['input', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='You are an expert senior software engineer. You are responsible for answering questions about LangChain. Use functions to consult the documentation before answering.', template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='{input}', template_format='f-string', validate_template=True), additional_kwargs={}), MessagesPlaceholder(variable_name='agent_scratchpad')])

In [ ]:
import functools
from langchain.smith import (
    arun_on_dataset,
    run_on_dataset, 
)

chain_results = run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=functools.partial(agent_factory, prompt=prompt),
    evaluation=evaluation_config,
    verbose=True,
    client=client,
    project_name=f"runnable-agent-test-5d466cbc-{unique_id}",
    tags=["testing-notebook", "prompt:5d466cbc"],  # Optional, adds a tag to the resulting chain runs
)

# Sometimes, the agent will error due to parsing issues, incompatible tool inputs, etc.
# These are logged as warnings here and captured as errors in the tracing UI.

### Review the test results

You can review the test results tracing UI below by clicking the URL in the output above or navigating to the "Testing & Datasets" page in LangSmith  **"f"langsmith-docs-dataset-{unique_id}"*"** dataset. 

This will show the new runs and the feedback logged from the selected evaluators. You can also explore a summary of the results in tabular format below.

In [ ]:
chain_results.to_dataframe()

### (Optional) Compare to another prompt

Now that we have our test run results, we can make changes to our agent and benchmark them. Let's try this again with a different prompt and see the results.

In [ ]:
candidate_prompt = hub.pull("wfh/langsmith-agent-prompt:798e7324")

chain_results = run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=functools.partial(agent_factory, prompt=candidate_prompt),
    evaluation=evaluation_config,
    verbose=True,
    client=client,
    project_name=f"runnable-agent-test-5d466cbc-{unique_id}",
    tags=["testing-notebook", "prompt:5d466cbc"],  # Optional, adds a tag to the resulting chain runs
)

## Exporting datasets and runs

LangSmith lets you export data to common formats such as CSV or JSONL directly in the web app. You can also use the client to fetch runs for further analysis, to store in your own database, or to share with others. Let's fetch the run traces from the evaluation run.

**Note: It may be a few moments before all the runs are accessible.**

In [ ]:
runs = client.list_runs(project_name=chain_results["project_name"], execution_order=1)

In [ ]:
# After some time, these will be populated.
client.read_project(project_name=chain_results["project_name"]).feedback_stats

## Conclusion

Congratulations! You have succesfully traced and evaluated an agent using LangSmith!

This was a quick guide to get started, but there are many more ways to use LangSmith to speed up your developer flow and produce better results.

For more information on how you can get the most out of LangSmith, check out [LangSmith documentation](https://docs.smith.langchain.com/), and please reach out with questions, feature requests, or feedback at [support@langchain.dev](mailto:support@langchain.dev).